[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/itmorn/AI.handbook/blob/main/DL/torch/nn/Normalization/InstanceNorm.ipynb)

# InstanceNorm2d
在4D输入(NCHW)上应用Layer Normalization，论文参考[Instance Normalization: The Missing Ingredient for Fast Stylization](https://arxiv.org/abs/1607.08022)

**定义**：  
torch.nn.InstanceNorm2d(num_features, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False, device=None, dtype=None)

**公式**：  
$$y = \frac{x - \mathrm{E}[x]}{ \sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta$$

和BN类似，参数可参考BN的讲解

# 图解train模式下的前向传播过程

<p align="center">
<a href="https://raw.githubusercontent.com/itmorn/AI.handbook/main/DL/torch/nn/Normalization/imgs/InstanceNorm.svg">
<img src="./imgs/InstanceNorm.svg"
    width="2000" /></a></p>


In [2]:
# 调包计算
import torch
import torch.nn as nn

input1 = torch.tensor([
    [
        [[1, 6],
         [9, 4]],
        [[12, 18],
         [13, 11]]],
    [
        [[2, 7],
         [3, 8]],
        [[19, 17],
         [15, 11]]
    ]
], dtype=torch.float32)
print("input1:\n", input1,"\n")

N, C, H, W = 2, 2, 2, 2

m = nn.InstanceNorm2d(num_features=C, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)

output = m(input1)
print("output:\n", output,"\n") # 结果和手工计算一致

input1:
 tensor([[[[ 1.,  6.],
          [ 9.,  4.]],

         [[12., 18.],
          [13., 11.]]],


        [[[ 2.,  7.],
          [ 3.,  8.]],

         [[19., 17.],
          [15., 11.]]]]) 

output:
 tensor([[[[-1.3720,  0.3430],
          [ 1.3720, -0.3430]],

         [[-0.5571,  1.6713],
          [-0.1857, -0.9285]]],


        [[[-1.1767,  0.7845],
          [-0.7845,  1.1767]],

         [[ 1.1832,  0.5071],
          [-0.1690, -1.5213]]]]) 



In [3]:
# 手工计算
import torch

# 这里涉及到组的计算，编程实现比较麻烦，展示起来不直观，这里就手工计算第1组，证明正确性即可。
input1 = torch.tensor([
    [
        [[1, 6],
         [9, 4]],
        [[12, 18],
         [13, 11]]],
    [
        [[2, 7],
         [3, 8]],
        [[19, 17],
         [15, 11]]
    ]
], dtype=torch.float32)

print("input1:\n", input1, "\n")

# 第1步：求均值和方差：
VarX, EX = torch.var_mean(input1, dim=(2,3),unbiased=False)  # NCHW
print("Ex:\n", EX, "\n")
print("VarX:\n", VarX, "\n")

# 第2步：减去均值：
result2 = input1-EX
print("input1-Ex:\n", result2, "\n")

# 第3步：求sqrt(VarX+eps)：
eps = 1e-5
result3 = torch.sqrt(VarX+eps)
print("sqrt(VarX+eps):\n", result3, "\n")

# 第4步：第2步的结果/第3步的结果，完成batch内的数据规范化:
result4 = result2/result3
print("(input1-Ex)/sqrt(VarX+eps):\n", result4, "\n")

# 第5步：使用γ=1，β=0 进行再校正：
γ = 1
β = 0
result5 = result4 * γ + β
print("[(input1-Ex)/sqrt(VarX+eps)] * γ + β:\n", result5, "\n") # 结果和图上一致


input1:
 tensor([[[[ 1.,  6.],
          [ 9.,  4.]],

         [[12., 18.],
          [13., 11.]]],


        [[[ 2.,  7.],
          [ 3.,  8.]],

         [[19., 17.],
          [15., 11.]]]]) 

Ex:
 tensor([[ 5.0000, 13.5000],
        [ 5.0000, 15.5000]]) 

VarX:
 tensor([[8.5000, 7.2500],
        [6.5000, 8.7500]]) 

input1-Ex:
 tensor([[[[ -4.0000,  -7.5000],
          [  4.0000, -11.5000]],

         [[  7.0000,   4.5000],
          [  8.0000,  -4.5000]]],


        [[[ -3.0000,  -6.5000],
          [ -2.0000,  -7.5000]],

         [[ 14.0000,   3.5000],
          [ 10.0000,  -4.5000]]]]) 

sqrt(VarX+eps):
 tensor([[2.9155, 2.6926],
        [2.5495, 2.9580]]) 

(input1-Ex)/sqrt(VarX+eps):
 tensor([[[[-1.3720, -2.7854],
          [ 1.5689, -3.8877]],

         [[ 2.4010,  1.6713],
          [ 3.1379, -1.5213]]],


        [[[-1.0290, -2.4140],
          [-0.7845, -2.5355]],

         [[ 4.8020,  1.2999],
          [ 3.9223, -1.5213]]]]) 

[(input1-Ex)/sqrt(VarX+eps)] * γ + β:
 te